In [1]:
from googleapiclient.discovery import build
import pandas as pd
import numpy as np
import seaborn as sns
import os
from dotenv import load_dotenv
from IPython.display import JSON

load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [2]:
api_key = os.getenv('YOUTUBE_API_KEY')


In [3]:
channel_ids = [
    'UCo07fumrTn1w4AxcU4j_uDw', #Times music
    'UCq-Fj5jknLsUf-MWSy4_brA', # T-series
    'UC56gTxNs4f9xZ7Pa2i5xNzg', # Sony Music
    'UCFFbwnve3yF62-tVXkTyHqg'  # Zee Music Company
    
]

In [4]:
youtube = build('youtube' , 'v3' , developerKey=api_key)

In [5]:
def get_channels_stats(youtube , channel_ids):
    all_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id= ','.join(channel_ids)
    )
    response = request.execute()

    for i in range(len(response['items'])):
        data = dict(
            Channel_name = response['items'] [i] ['snippet']['title'],
            Subscribers =  response['items'] [i] ['statistics']['subscriberCount'],
            Views =  response['items'] [i] ['statistics']['viewCount'],
            Videos =  response['items'] [i] ['statistics']['videoCount'],
            playlist_id = response['items'] [i] ['contentDetails']['relatedPlaylists']['uploads'] )

        all_data.append(data)
        

    return all_data

In [6]:
channel_stats =  get_channels_stats(youtube , channel_ids)

In [17]:
channel_stats

[{'Channel_name': 'Times Music',
  'Subscribers': '16900000',
  'Views': '6021737246',
  'Videos': '2912',
  'playlist_id': 'UUo07fumrTn1w4AxcU4j_uDw'},
 {'Channel_name': 'Zee Music Company',
  'Subscribers': '114000000',
  'Views': '74201548605',
  'Videos': '13443',
  'playlist_id': 'UUFFbwnve3yF62-tVXkTyHqg'},
 {'Channel_name': 'T-Series',
  'Subscribers': '286000000',
  'Views': '284330478025',
  'Videos': '22657',
  'playlist_id': 'UUq-Fj5jknLsUf-MWSy4_brA'},
 {'Channel_name': 'Sony Music India',
  'Subscribers': '63700000',
  'Views': '33500279526',
  'Videos': '6228',
  'playlist_id': 'UU56gTxNs4f9xZ7Pa2i5xNzg'}]

In [7]:
channel_data = pd.DataFrame(channel_stats)

In [8]:
channel_data

,Channel_name,Subscribers,Views,Videos,playlist_id
0,Times Music,16900000,6021737246,2912,UUo07fumrTn1w4AxcU4j_uDw
1,Zee Music Company,114000000,74201548605,13443,UUFFbwnve3yF62-tVXkTyHqg
2,T-Series,286000000,284330478025,22657,UUq-Fj5jknLsUf-MWSy4_brA
3,Sony Music India,63700000,33500279526,6228,UU56gTxNs4f9xZ7Pa2i5xNzg


In [20]:
playlist_ids = []

def get_playlists():
    global playlist_ids
    playlist_ids = [channel['playlist_id'] for channel in channel_stats if 'playlist_id' in channel]

get_playlists()
print(playlist_ids)


['UUo07fumrTn1w4AxcU4j_uDw', 'UUFFbwnve3yF62-tVXkTyHqg', 'UUq-Fj5jknLsUf-MWSy4_brA', 'UU56gTxNs4f9xZ7Pa2i5xNzg']


In [21]:
def get_video_ids(youtube, playlist_ids):
    all_video_ids = []  

    for playlist_id in playlist_ids:  
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50
        )
        response = request.execute()

        video_ids = [item['contentDetails']['videoId'] for item in response.get('items', [])]
        all_video_ids.extend(video_ids)  # Add videos from the first request

        next_page_token = response.get('nextPageToken')

        while next_page_token:  # Fetch remaining pages if available
            request = youtube.playlistItems().list(
                part="contentDetails",
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token
            )
            response = request.execute()

            video_ids = [item['contentDetails']['videoId'] for item in response.get('items', [])]
            all_video_ids.extend(video_ids)  # Append new video IDs

            next_page_token = response.get('nextPageToken')

    return all_video_ids  


In [23]:
video_ids = get_video_ids(youtube ,playlist_ids)

In [24]:
len(video_ids)

42576

In [25]:
def get_video_details(youtube, video_ids, channel_name):
    all_video_stats = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part='snippet,statistics',
            id=','.join(video_ids[i:i+50])
        )

        response = request.execute()

        for video in response['items']:
            video_stats = dict(
                Channel_Name=channel_name,  # Add channel name
                Title=video['snippet']['title'],
                Date_Released=video['snippet']['publishedAt'],
                Views=video['statistics'].get('viewCount', 0),
                Likes=video['statistics'].get('likeCount', 0),
                Comments=video['statistics'].get('commentCount', 0)
            )

            all_video_stats.append(video_stats)

    return all_video_stats


In [28]:
all_videos = []

for channel in channel_stats:
    playlist_id = channel['playlist_id']
    channel_name = channel['Channel_name']  # Extract channel name

    video_ids = get_video_ids(youtube, playlist_id)  # Fetch video IDs
    video_details = get_video_details(youtube, video_ids, channel_name)  # Fetch details

    all_videos.extend(video_details)  # Store all results


In [29]:
video_data = pd.DataFrame(all_videos)

In [30]:
video_data

,Channel_Name,Title,Date_Released,Views,Likes,Comments
0,Times Music,Sunke (Full Audio) | Khan Bhaini | Teji Sandhu...,2025-02-08T11:30:08Z,1781,96,12
1,Times Music,✨ Lose yourself in the melody of love! 💖🎵 Dil ...,2025-02-08T10:02:47Z,1702,49,2
2,Times Music,🎉 Let the magic of Khali Peeli Baatein brighte...,2025-02-08T09:49:27Z,2033,47,1
3,Times Music,Peace Lily (Synth-Pop Mix) | Knockwell | @Aar...,2025-02-08T04:30:01Z,1458,30,4
4,Times Music,Gustakhiyaan (House Mix) | Knockwell | Raghav ...,2025-02-07T11:30:41Z,2298,55,3
...,...,...,...,...,...,...
42571,Sony Music India,Crook - Emraan Hashmi's Brand New Supercool Pr...,2010-09-30T08:26:29Z,4303,65,8
42572,Sony Music India,Crook - Emraan Hashmi's Brand New Supercool Pr...,2010-09-30T08:18:41Z,6642,72,8
42573,Sony Music India,Crook - Emraan Hashmi's Brand New Supercool Pr...,2010-09-30T08:06:23Z,9404,136,15
42574,Sony Music India,You Are My Soniya - K3G | Kareena Kapoor | Hri...,2009-10-05T22:07:36Z,244345,783,11


In [32]:
video_data['Date_Released'] = pd.to_datetime(video_data['Date_Released']).dt.date
video_data['Views'] = pd.to_numeric(video_data['Views'])
video_data['Likes'] = pd.to_numeric(video_data['Likes'])
video_data['Comments'] = pd.to_numeric(video_data['Comments'])
video_data

,Channel_Name,Title,Date_Released,Views,Likes,Comments
0,Times Music,Sunke (Full Audio) | Khan Bhaini | Teji Sandhu...,2025-02-08,1781,96,12
1,Times Music,✨ Lose yourself in the melody of love! 💖🎵 Dil ...,2025-02-08,1702,49,2
2,Times Music,🎉 Let the magic of Khali Peeli Baatein brighte...,2025-02-08,2033,47,1
3,Times Music,Peace Lily (Synth-Pop Mix) | Knockwell | @Aar...,2025-02-08,1458,30,4
4,Times Music,Gustakhiyaan (House Mix) | Knockwell | Raghav ...,2025-02-07,2298,55,3
...,...,...,...,...,...,...
42571,Sony Music India,Crook - Emraan Hashmi's Brand New Supercool Pr...,2010-09-30,4303,65,8
42572,Sony Music India,Crook - Emraan Hashmi's Brand New Supercool Pr...,2010-09-30,6642,72,8
42573,Sony Music India,Crook - Emraan Hashmi's Brand New Supercool Pr...,2010-09-30,9404,136,15
42574,Sony Music India,You Are My Soniya - K3G | Kareena Kapoor | Hri...,2009-10-05,244345,783,11


In [ ]:
video_data.to_csv("csv/MultipleChannel.csv", index=False)